In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tkinter import *
import os

In [45]:
try:
    if not os.path.exists('photos'):
        os.makedirs('photos')
except OSError:
    print("Error in creating directory")
    

In [46]:
def imagetake():
    facedetection=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    def face_extraction(img):
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        face=facedetection.detectMultiScale(gray,1.3,5)
        if face is ():
            return None
        for (x,y,w,h) in face:
            cropped=img[y:y+h,x:x+w]
        return cropped


    cap=cv2.VideoCapture(0)
    count=0
    ret,frame=cap.read()
    while ret:
        ret, frame=cap.read()
        #gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        if face_extraction(frame) is not None:
            count+=1
            face=cv2.resize(face_extraction(frame),(200,200))
            face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            path='photos//'+str(count)+'.jpg'
            cv2.imwrite(path,face)
            cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow('face',face)
        else:
            print('face not found')
            pass

        if cv2.waitKey(1)==13 or count==100:
            break

    cap.release()
    cv2.destroyAllWindows()



In [47]:
def imagedetector():
    Training_data,Labels=[],[]
    path='photos//'
    for i in range(1,101):
        paths=path+str(i)+'.jpg'
        images=cv2.imread(paths,cv2.IMREAD_GRAYSCALE)
        Training_data.append(np.asarray(images,dtype=np.uint8))
        Labels.append(i)
    Labels=np.asarray(Labels,dtype=np.int32)
    model=cv2.face.LBPHFaceRecognizer_create()
    model.train(np.asarray(Training_data),np.asarray(Labels))
    facedetection1=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
   
    def face_detector(img):
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        face=facedetection1.detectMultiScale(gray,1.3,5)
        print(face)
        if face is ():
            return img,[]
        for (x,y,w,h) in face:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
            roi=img[y:y+h,x:x+w]
            roi=cv2.resize(roi,(200,200))
        return img ,roi
    cap=cv2.VideoCapture(0)
    ret,frame=cap.read()
    while ret:
        ret,frame=cap.read()
        img,face=face_detector(frame)

        try:
            face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            result=model.predict(face)
            if result[1]<500:
                confidence=int(100*(1-(result[1])/300))
                display_s=str(confidence)+'% confidence its user'
            cv2.putText(img,display_s,(100,120),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            if confidence > 85:
                cv2.putText(img,'UNLOCKED',(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                cv2.imshow('Face Cropper',img)
            else:
                cv2.putText(img,'LOCKED',(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                cv2.imshow('Face Cropper',img)
        except:
            cv2.putText(img,'NO face found',(220,120),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow('Face Cropper',img)
            pass
        if cv2.waitKey(1)==13:
            break
    cap.release()
    cv2.destroyAllWindows()



In [48]:
class app:
    def __init__(self,master):
        self.master=master
        self.ft=Frame(master,width=800,height=720,bg='#353535')
        self.ft.pack()
        self.heading=Label(self.ft,text='FACE RECOGNITION',font=('arial 40 bold'),fg='black',bg='#0ffff5')
        self.heading.place(x=130,y=0)
        self.image=Button(self.ft,text='YOUR IMAGES',width=52,height=2,bg='steelblue',command=imagetake)
        self.image.place(x=220,y=150)
        self.detect=Button(self.ft,text='FACE DETECTOR',width=52,height=2,bg='steelblue',command=imagedetector)
        self.detect.place(x=220,y=200)

root=Tk()
app(root)
#root.resizable(False,False)
root.mainloop()